### Progetto 8b
Luca Maria Lauricella 519997

Valerio Marini 512489

Repository del progetto:
https://github.com/lauriluca99/TGW-3D.jl

Documentazione del progetto:
https://lauriluca99.github.io/TGW-3D.jl

In [1]:
using BenchmarkTools
using SparseArrays
using NearestNeighbors
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar=LinearAlgebraicRepresentation

const Points = Matrix
const ChainOp = SparseArrays.SparseMatrixCSC{Int8,Int}

SparseMatrixCSC{Int8, Int64}

    function spatial_arrangement(
            V::Points, 
            copEV::ChainOp, 
            copFE::ChainOp; 
            [multiproc::Bool])
Calcola la disposizione sulle cellule complesse 2-skeleton date	in 3D.
														
Un complesso cellulare è disposto quando l'intersezione di ogni coppia di celle
del complesso è vuota e l'unione di tutte le celle rappresenta l'intero spazio Euclideo.
La funzione ritorna la piena disposizione complessa come una lista di vertici V e una catena di lati EV, FE, CF.
#### Argomenti addizionali:
- `multiproc::Bool`: Esegue la computazione in modalità parallela. Di Default a `false`.

# CODICE COMPLETO

In [2]:
function frag_face(V::Points, EV::ChainOp, FE::ChainOp, 
    sp_idx::Vector{Vector{Int64}}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Points = V[sigmavs, :]
    sEV::ChainOp = EV[FE[sigma, :].nzind, sigmavs]
    M::Points = Lar.Arrangement.submanifold_mapping(sV)
    @views tV::Points = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Points, tmpEV::ChainOp = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    @views sV = sV[:, 1:2]
    nV::Points, nEV::ChainOp, nFE::ChainOp = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    @views nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [3]:
function filter_fn(face)
    visited = []
    verts = []
    map(e->verts = union(verts, collect(e)), face)
    verts = Set(verts)
 
    if !(verts in visited)
        push!(visited, verts)
        return true
    end
    return false
end

filter_fn (generic function with 1 method)

In [4]:
function merge_vertices(V::Points, EV::ChainOp, FE::ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #questo for non può essere parallelizzato
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #questo for non può essere parallelizzato
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #questo ciclo non può essere parallelizzato
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    #questo ciclo può essere parallelizzato ma non cambia il tempo per un numero di edge piccolo
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
    @inbounds nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Points(nV), nEV, nFE
 end

merge_vertices (generic function with 2 methods)

In [5]:
function removeinnerloops(g::Int64, nFE::ChainOp)
	FE::Vector{Vector{Int64}} = Lar.cop2lar(nFE)
	nFE::ChainOp = Lar.lar2cop(FE[1:end-g])
	return nFE
end

removeinnerloops (generic function with 1 method)

In [6]:
function spatial_arrangement_1(
    V::Points,
    copEV::ChainOp,
    copFE::ChainOp, multiproc::Bool=false)

# spaceindex computation
FV = Lar.compute_FV( copEV, copFE )
model = (convert(Points,V'), FV)
sp_idx = Lar.spaceindex(model)

# initializations
fs_num = size(copFE, 1)
rV = Array{Float64,2}(undef,0,3)
rEV = SparseArrays.spzeros(Int8,0,0)
rFE = SparseArrays.spzeros(Int8,0,0)
# sequential (iterative) processing of face fragmentation
     for sigma in 1:fs_num
        #print(sigma, "/", fs_num, "\r")
        nV, nEV, nFE = frag_face(V, copEV, copFE, sp_idx, sigma)
        #nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
        nV = convert(Points, nV)
        rV, rEV, rFE = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
        #rV=a;  rEV=b;  rFE=c
    end
# merging of close vertices, edges and faces (3D congruence)
rV, rEV, rFE = merge_vertices(rV, rEV, rFE)
return rV, rEV, rFE
end

spatial_arrangement_1 (generic function with 2 methods)

In [7]:
function face_angle(e::Int, f::Int)

    edge_vs = EV[e, :].nzind

    t = findfirst(x->edge_vs[1] in x && edge_vs[2] in x, triangulated_faces[f])

    v1 = normalize(V[edge_vs[2], :] - V[edge_vs[1], :])

    if abs(v1[1]) > abs(v1[2])
        invlen = 1. / sqrt(v1[1]*v1[1] + v1[3]*v1[3])
        v2 = [-v1[3]*invlen, 0, v1[1]*invlen]
    else
        invlen = 1. / sqrt(v1[2]*v1[2] + v1[3]*v1[3])
        v2 = [0, -v1[3]*invlen, v1[2]*invlen]
    end

    v3 = cross(v1, v2)

    M = reshape([v1; v2; v3], 3, 3)

    triangle = triangulated_faces[f][t]
    third_v = setdiff(triangle, edge_vs)[1]
    vs = V[[edge_vs..., third_v], :]*M

    v = vs[3, :] - vs[1, :]
    angle = atan(v[2], v[3])
    return angle
end

face_angle (generic function with 1 method)

In [8]:
function minimal_3cycles(V::Points, EV::ChainOp, FE::ChainOp)

	triangulated_faces = Array{Any, 1}(undef, FE.m)

    #EF = FE'
    EF::ChainOp = convert(ChainOp, LinearAlgebra.transpose(FE))
	FC::ChainOp = Lar.Arrangement.minimal_cycles(face_angle, true)(V, EF)

	#FC'
    return -convert(ChainOp, LinearAlgebra.transpose(FC))
end

minimal_3cycles (generic function with 1 method)

In [9]:
function spatial_arrangement_2(
    rV::Points,
    rcopEV::ChainOp,
    rcopFE::ChainOp, 
    multiproc::Bool=false)

#rcopCF = Lar.build_copFC(rV, rcopEV, rcopFE)  ######
rcopCF::ChainOp = minimal_3cycles(rV, rcopEV, rcopFE)

return rV, rcopEV, rcopFE, rcopCF
end

spatial_arrangement_2 (generic function with 2 methods)

In [10]:
function spatial_arrangement(
    V::Points, # by rows
    copEV::ChainOp,
    copFE::ChainOp, 
    multiproc::Bool=false)

# face subdivision
rV::Points, rcopEV::ChainOp, rcopFE::ChainOp = spatial_arrangement_1( V,copEV,copFE,multiproc )

bicon_comps = Lar.Arrangement.biconnected_components(rcopEV)

rV, rEV::ChainOp, rFE::ChainOp, rCF::ChainOp = spatial_arrangement_2(rV, rcopEV, rcopFE)
end

spatial_arrangement (generic function with 2 methods)

In [11]:
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
]

EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
])

FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [12]:
#funzione originale della liberia LAR, non funziona su Windows
#@benchmark Lar.Arrangement.spatial_arrangement(Points(V),ChainOp(EV),ChainOp(FE),false)

# Ottimizzazione
Stabilizzando i tipi delle variabili e parallelizzando le funzioni merge_vertices e frag_face abbiamo ottenuto un codice che si comporta come una versione più veloce del codice precedente risparmiando circa un 40% del tempo di esecuzione.

In [13]:
#funzione ottimizzata
@benchmark spatial_arrangement(Points(V), ChainOp(EV), ChainOp(FE), false)

BenchmarkTools.Trial: 1478 samples with 1 evaluation.
 Range (min … max):  2.392 ms … 30.884 ms  ┊ GC (min … max):  0.00% … 81.14%
 Time  (median):     2.843 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.375 ms ±  2.406 ms  ┊ GC (mean ± σ):  11.76% ± 13.74%

  ███▆▄▄▃▂                                                    
  ██████████▅▇▆▇▅▅▄▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▁▁▁▁▄▄▁▅▄▅▅▅▆▅▁▇▄▅▅▆ █
  2.39 ms      Histogram: log(frequency) by time     14.6 ms <

 Memory estimate: 1.59 MiB, allocs estimate: 28812.

In [51]:
V = Float64[0.3229956 0.8459234 0.3091824 0.8321101 0.3233606 0.8462884 0.3095474 0.8324751 0.822632 0.8740866 0.8719075 0.923362 0.9865184 1.0379729 1.0357938 1.0872484; 
            0.5770385 0.5632252 0.0548407 0.0410274 0.6046603 0.590847 0.0824624 0.0686491 0.4321656 0.4814411 0.591786 0.6410615 0.368702 0.4179775 0.5283224 0.5775979; 
            0.0769021 0.0772671 0.1045238 0.1048888 0.5992825 0.5996475 0.6269042 0.6272692 0.1754594 0.3393458 0.1119958 0.2758822 0.1430863 0.3069727 0.0796228 0.2435091]
CV = [[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16]]
FV = [[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 5, 6], [3, 4, 7, 8], [1, 3, 5, 7], [2, 4, 6, 8], [9, 10, 11, 12], [13, 14, 15, 16], [9, 10, 13, 14], [11, 12, 15, 16], [9, 11, 13, 15], [10, 12, 14, 16]]
EV = [[1, 2], [3, 4], [5, 6], [7, 8], [1, 3], [2, 4], [5, 7], [6, 8], [1, 5], [2, 6], [3, 7], [4, 8], [9, 10], [11, 12], [13, 14], [15, 16], [9, 11], [10, 12], [13, 15], [14, 16], [9, 13], [10, 14], [11, 15], [12, 16]]

FE = Lar.boundary_1(vec(permutedims(EV)))
EV = transpose(Lar.boundary_1(EV))
@show EV
spatial_arrangement(Points(V), ChainOp(EV), ChainOp(FE))

EV = Int8[-1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 -1 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 -1 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 -1 1 0 0 0 0 0 0 0 0; -1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 -1 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 -1 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 -1 0 1 0 0 0 0 0 0 0 0; -1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0; 0 -1 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 -1 0 0 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 -1 0 0 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 -1 1 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 -1 1 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 -1 1 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 1; 0 0 0 0 0 0 0 0 -1 0 1 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 -1 0 1 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 1 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 1; 0 0 0 0 0 0 0 0 -1 0 0 0 1 0 0 0; 0 0 0 0 0 0 0 0 0 -1 0 0 0 1 0 0; 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 1 0; 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 1]


LoadError: BoundsError: attempt to access 16×3 Matrix{Float64} at index [1:16, [4]]

In [83]:
V = Float64[0.3229956 0.8459234 0.3091824 0.8321101 0.3233606 0.8462884 0.3095474 0.8324751 0.822632 0.8740866 0.8719075 0.923362 0.9865184 1.0379729 1.0357938 1.0872484; 
            0.5770385 0.5632252 0.0548407 0.0410274 0.6046603 0.590847 0.0824624 0.0686491 0.4321656 0.4814411 0.591786 0.6410615 0.368702 0.4179775 0.5283224 0.5775979; 
            0.0769021 0.0772671 0.1045238 0.1048888 0.5992825 0.5996475 0.6269042 0.6272692 0.1754594 0.3393458 0.1119958 0.2758822 0.1430863 0.3069727 0.0796228 0.2435091]
CV = [[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16]]
FV = [[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 5, 6], [3, 4, 7, 8], [1, 3, 5, 7], [2, 4, 6, 8], [9, 10, 11, 12], [13, 14, 15, 16], [9, 10, 13, 14], [11, 12, 15, 16], [9, 11, 13, 15], [10, 12, 14, 16]]
EV = [[1, 2], [3, 4], [5, 6], [7, 8], [1, 3], [2, 4], [5, 7], [6, 8], [1, 5], [2, 6], [3, 7], [4, 8], [9, 10], [11, 12], [13, 14], [15, 16], [9, 11], [10, 12], [13, 15], [14, 16], [9, 13], [10, 14], [11, 15], [12, 16]]

#FE = FV x EV'

FE = u_coboundary_1(FV::Cells, EV::Cells, true)::ChainOp
FE = convert(ChainOp, FE');
spatial_arrangement(Points(V), ChainOp(EV), FE)